# Traveling salesman problem

## Installation

If you run this notebook on Colab, install the following dependencies first:

In [ ]:
# !pip install pyvrp folium scikit-learn

If you run this notebook locally, make sure to install [uv](https://docs.astral.sh/uv/getting-started/installation/) first.

## Data

In [ ]:
DATA = """Name,Latitude,Longitude
Amsterdam,52.3676,4.9041
Rotterdam,51.9225,4.47917
The Hague (Den Haag),52.0705,4.3007
Utrecht,52.0907,5.1214
Eindhoven,51.4416,5.4697
Groningen,53.2194,6.5665
Tilburg,51.5555,5.0913
Almere,52.3508,5.2647
Breda,51.5719,4.7683
Nijmegen,51.8425,5.8528
Enschede,52.2215,6.8937
Haarlem,52.3874,4.6462
Arnhem,51.9851,5.8987
Zaanstad,52.4571,4.7510
's-Hertogenbosch (Den Bosch),51.6992,5.3040
Amersfoort,52.1561,5.3878
Haarlemmermeer,52.3014,4.6910
Maastricht,50.8514,5.6900
Leiden,52.1601,4.4970
Dordrecht,51.8133,4.6901
Zoetermeer,52.0575,4.4931
Zwolle,52.5168,6.0830
Leeuwarden,53.2012,5.8086
Alkmaar,52.6324,4.7534
Emmen,52.7850,6.8976
Westland,52.0240,4.2000
Delft,52.0116,4.3571
Deventer,52.2550,6.1639
Helmond,51.4793,5.6570
Heerlen,50.8882,5.9795
Hilversum,52.2292,5.1669
Oss,51.7650,5.5183
Sittard-Geleen,50.9950,5.8667
Hengelo,52.2653,6.7931
Purmerend,52.5050,4.9597
Roosendaal,51.5308,4.4653
Schiedam,51.9167,4.4000
Lelystad,52.5185,5.4714
Almelo,52.3670,6.6683
Spijkenisse,51.8450,4.3292
Hoorn,52.6425,5.0597
Vlaardingen,51.9125,4.3417
Venlo,51.3704,6.1724
Nieuwegein,52.0292,5.0806
Gouda,52.0116,4.7100
Assen,52.9925,6.5642
Veenendaal,52.0286,5.5589
Bergen op Zoom,51.4939,4.2917
Capelle aan den IJssel,51.9292,4.5778
Katwijk,52.2039,4.3986
Zeist,52.0906,5.2333"""

## Imports

In [ ]:
import pandas as pd
import folium
from IPython.display import display
import io
from dataclasses import dataclass
import random
import numpy as np
from copy import copy

## Utility functions

In [ ]:
def draw(df, tour=None):
    """
    Draws a map with location markers and lines connecting points according
    to a specified tour.

    Parameters:
    -----------
    df: pd.DataFrame
        DataFrame containing at minimum 'Latitude' and 'Longitude' columns.
    tour: list
        List of indices specifying the order to connect points.
        If None, no lines are drawn.
    """
    center_lat = 52.1326
    center_lon = 5.2913
    zoom_start = 7
    width = 400
    height = 420
    point_radius = 0.1
    point_color = "red"
    line_color = "blue"
    line_weight = 2

    fmap = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=zoom_start,
        tiles="Cartodb positron",
        width=width,
        height=height,
    )

    for _, row in df.iterrows():
        popup = f"{row['Name']}" if "Name" in df.columns else None
        folium.CircleMarker(
            location=[row["Latitude"], row["Longitude"]],
            radius=point_radius,
            popup=popup,
            color=point_color,
            fill=True,
            fill_color=point_color,
            fill_opacity=0.6,
        ).add_to(fmap)

    if tour is not None:
        coordinates = []
        for idx in tour:
            row = df.iloc[idx]
            coordinates.append([row["Latitude"], row["Longitude"]])

        if tour[0] != tour[-1]:
            coordinates.append(coordinates[0])

        folium.PolyLine(
            locations=coordinates,
            color=line_color,
            weight=line_weight,
            opacity=0.7,
        ).add_to(fmap)

    return fmap

In [ ]:
df = pd.read_csv(io.StringIO(DATA))

# Example usage:
tour = [0, 2, 1, 3]  # Example: visit in this order, returning to start
draw(df, tour=tour)

### Haversine distance

In [ ]:
def haversine(coords: list[tuple[int, int]]) -> np.ndarray:
    """
    Converts coordinates to a Haversine distance matrix (in meters).
    """

    from sklearn.metrics.pairwise import haversine_distances
    from math import radians

    RADIUS_EARTH = 6371000 # meters

    coords_rad = np.array(
        [[radians(lat), radians(lon)] for lat, lon in coords]
    )
    dist_matrix = haversine_distances(coords_rad)
    dist_meters = (dist_matrix * RADIUS_EARTH).round() # rads to meters
    return dist_meters

### ProblemData

In [ ]:
@dataclass
class ProblemData:
    distances: list[list[int]]

    @property
    def num_clients(self):
        return len(self.distances)

### Cost function

In [ ]:
def cost(tour, data):
    """
    Computes the cost of a tour.
    """
    nodes = tour + [tour[0]]
    frm = nodes[:-1]
    to = nodes[1:]
    return data.distances[frm, to].sum()

## Nearest neighbor

In [ ]:
def nearest_neighbor(data: ProblemData) -> list[int]:
    current = 0
    tour = [current]

    while len(tour) < data.num_clients:
        best_dist = None
        best_neighbour = None 

        for client in range(data.num_clients):
            if client not in tour:
                dist = data.distances[current, client]
                if best_dist is None or dist < best_dist:
                    best_dist = dist 
                    best_neighbour = client
                
        neighbor = best_neighbour
        tour.append(neighbor)
        current = neighbor

    return tour

In [ ]:
coords = df[["Latitude", "Longitude"]].values
data = ProblemData(haversine(coords))

In [ ]:
tour = nearest_neighbor(data)
cost(tour, data)

In [ ]:
draw(df, tour=tour)

## Large neighborhood search

In [ ]:
def large_neighborhood_search(
    data: ProblemData,
    num_iterations: int,
    num_removals: int = 5,
) -> list[int]:
    curr_tour = nearest_neighbor(data)
    curr_cost = cost(curr_tour, data)

    for _ in range(num_iterations):
        new_tour = copy(curr_tour)
        removed = ... # remove random clients

        for client in removed: 
			new_tour = ... # re-insert removed clients

        new_cost = cost(new_tour, data)
        if new_cost < curr_cost:
            curr_cost = new_tour
            curr_tour = cand_tour

    return curr_tour

In [ ]:
lns = large_neighborhood_search(data, num_iterations=10000, num_removals=5)
cost(lns, data)

In [ ]:
draw(df, tour=lns)